# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [53]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [54]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: @bjsmjil


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key) = JQNEcKGAjiyDLocQtNAMMVKeI
    1. Consumer Secret (API Secret) = 7ERqHyCg6saIEYhOPcjrDHBH6loH54FNDOCUyYYIIytaeoLRnM

1. Mais abaixo, gere um Token e anote também:
    1. Access Token = 903304086061187073-oUyvgEs4T0bxP8HQmks7OfqJs3z0H1F
    1. Access Token Secret = iJp2XXkfZhE2wz5p35IFR4ieMkjdc4lzstzLs40QydFPz

    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [55]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'Netshoes'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [ ]:
# função que remove pontuações
import string
def tira_pontuacao(texto):
    for punctuation in string.punctuation:
        texto = texto.replace(punctuation, '')
    return texto

In [ ]:
dados = pd.read_excel('Netshoes.xlsx')
relevantes_treinamento=[]
irrelevantes_treinamento=[]
dados.rename(columns={
                 'Unnamed: 1': 'Relevância'}, inplace=True)
dados["Treinamento"] = dados['Treinamento'].apply(tira_pontuacao)

# separando quanto a relevância
relevantes = dados[(dados['Relevância'] == 'S')]
irrelevantes = dados[(dados['Relevância'] == 'A')]

# Transformando siglas
dados.loc[(dados['Relevância']=='S'),'Relevância'] == 'Relevante'
dados.loc[(dados['Relevância']=='A'),'Relevância'] == 'Irelevante'

#limpando twitts

for i in range(len(dados)):
    linha = dados.iloc[i]['Treinamento']
    linha = linha.split()
    for j in range(len(linha)):
        linha[j]=linha[j].lower()
        linha[j]=linha[j].strip()
        linha[j] = linha[j].replace(".", "")
    dados.iloc[i]['Treinamento'] = linha


# selecionando palavras dos relevantes
for i in range(len(relevantes)):
    linha = relevantes.iloc[i]['Treinamento']
    linha = linha.split()
    for j in range(len(linha)):
        linha[j]=linha[j].lower()
        linha[j]=linha[j].strip()
        linha[j] = linha[j].replace(".", "")
        relevantes_treinamento.append(linha[j])
    relevantes.iloc[i]['Treinamento'] = linha
    
# selecionando palavras dos irrelevantes
for i in range(len(irrelevantes)):
    linha = irrelevantes.iloc[i]['Treinamento']
    linha = linha.split()
    for j in range(len(linha)):
        linha[j]=linha[j].lower()
        linha[j]=linha[j].strip()
        linha[j] = linha[j].replace(".", "")
        irrelevantes_treinamento.append(linha[j])
    irrelevantes.iloc[i]['Treinamento'] = linha
    

      


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [ ]:


# planilha teste

teste = pd.read_excel('Netshoes.xlsx', sheetname='Teste')

teste.rename(columns={
                 'Unnamed: 1': 'Relevância'}, inplace=True)
teste["Teste"] = teste['Teste'].apply(tira_pontuacao)
#limpando twitts
for i in range(len(teste)):
    linha = teste.iloc[i]['Teste']
    linha = linha.split()
    if 'netshoes' in linha:
        linha.remove('netshoes')
    if 'de' in linha:
        linha.remove('de')
    if 'o' in linha:
        linha.remove('o')
    if 'que' in linha:
        linha.remove('que')
    if 'em' in linha:
        linha.remove('em')
    if 'tem' in linha:
        linha.remove('tem')
    if 'no' in linha:
        linha.remove('no')
    if 'do' in linha:
        linha.remove('do')
    if 'um' in linha:
        linha.remove('um')
    if 'para' in linha:
        linha.remove('para')
    if 'é' in linha:    
        linha.remove('é')
    if 'e' in linha:    
        linha.remove('e')
    if 'a' in linha:    
        linha.remove('a')
    if 'pra' in linha:
        linha.remove('pra')

        
    for j in range(len(linha)):
        linha[j]=linha[j].lower()
        linha[j]=linha[j].strip()
        linha[j] = linha[j].replace(".", "")
    teste.iloc[i]['Teste'] = linha


# series relevantes
prob_wr = pd.Series(relevantes_treinamento).value_counts().to_dict()

# series irrelevantes
prob_wir = pd.Series(irrelevantes_treinamento).value_counts().to_dict()

# probabilidade da sentença ser relevante
todas_palavras = len(prob_wr)+len(prob_wir)
prob_relevante = len(prob_wr)/todas_palavras
prob_irrelevante = len(prob_wir)/todas_palavras    


# cálculo de probabilidade dos tweets ser relevante
prob_twitt_r=[]
for i in range(len(teste)):
    twitt= teste.iloc[i]['Teste']
    prob_twitt=1
    y=1
    for j in range(len(twitt)):
        for word in prob_wr:
            if twitt[j]==word:
                y=prob_wr[word]+1
                break
        prob_word=y/(len(prob_wr)+todas_palavras)
        prob_twitt*=prob_word
    prob_twitt_r.append(prob_twitt)

    

#cálculo de probabilidade dos tweets ser irrelevante    
prob_twitt_ir=[]
for i in range(len(teste)):
    twitt= teste.iloc[i]['Teste']
    prob_twitt=1
    y=1
    for j in range(len(twitt)):
        for word in prob_wir:
            if twitt[j]==word:
                y=prob_wir[word]+1
                break
        prob_word=y/(len(prob_wir)+todas_palavras)
        prob_twitt*=prob_word   
    prob_twitt_ir.append(prob_twitt)
        
                


In [ ]:
# Tomada de decisão

for index, row in teste.iterrows():
    i = teste.index.get_loc(index)
    if prob_twitt_r[i]*prob_relevante>prob_twitt_ir[i]*prob_irrelevante:
        teste.loc[index, 'Classificador'] = 'Relevante'


    if prob_twitt_r[i]*prob_relevante<prob_twitt_ir[i]*prob_irrelevante:
        teste.loc[index, 'Classificador'] = 'Irrelevante'

In [ ]:
Comparativo = pd.crosstab(teste['Relevância'],teste['Classificador'])
Comparativo

In [ ]:
PPF = (32/(17+32))*100
PPV = (17/(17+32))*100
PNV = (133/(18+133))*100
PNF = (18/(18+133))*100

print('Porcentagem de positivos falsos ',PPF,'\nPorcentagem de positivos verdadeiros',PPV,'\nPorcentagem de negativos verdadeiros',PNV,'\nPorcentagem de negativos falsos',PNF)

## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


       
       Após a coleta de dados feita pelo código presente neste documento, foi realizada uma classificação manual para determinar os tweets que são considerados relevantes ou não para alimentar o classificador automático de forma que ele aprenda as palavras que mais se adequam em cada tipo automaticamente.
        O classificador apresenta uma certa ambiguidade na forma como apresenta os dados pois por um lado classifica mal os tweets relevantes afinal a porcentagem de tweets classificados como relevantes e que não são é muito alta, assim como os que são relevantes têm uma porcentagem baixa, e por outro acaba classificando bem os tweets que não são relevantes, tendo uma alta porcentagem de acerto.
        O sistema do classificador não é preparado para interpretar sarcasmo ou dupla negação pois o método Naive-Bayes vizualiza cada palavra da senteça como uma probabilidade para rotular a frase como relevante ou não, sem analisar expressões ou sentimentos.
        Para além do projeto de classificar como sendo relevante ou não cada tweet o programa pode ser aperfeiçoado para apresentar uma classificação de humor do cliente, assim como definir se entre os tweets relevantes existe algum problema que é tratado de forma recorrente e isso exigirá um método que analise não somente palavras de uma frase mas também expressões. 
        O projeto é um indicador muito bom para relatar a irrelevância de uma frase por mesmo apresentando falhas na forma como avalia a relevância da mesma. Dado isto é satisfatória a forma como este trabalho opera.